In [52]:
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
import pandas as pd
import warnings
import pd_explain

warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', 999)
pd.set_option('display.max_columns', 999)

usetex mode requires TeX.


usetex-False


# Bank Churners Dataset


**CLIENTNUM** : Client number. Unique identifier for the customer holding the account

**Attrition_Flag :** Internal event (customer activity) variable - if the account is closed then 1 else 0

**Dependent_count :** Demographic variable - Number of dependents

**Total_Relationship_Count :** Total no. of products held by the customer

**Total_Revolving_Bal :** Total Revolving Balance on the Credit Card

**Avg_Open_To_Buy :** Open to Buy Credit Line (Average of last 12 months)

**Months_on_book :** Period of relationship with bank

**Months_Inactive_12_mon :** No. of months inactive in the last 12 months

**Contacts_Count_12_mon :** No. of Contacts in the last 12 months

**Total_Amt_Chng_Q4_Q1 :** Change in Transaction Amount (Q4 over Q1)

**Total_Trans_Amt :** Total Transaction Amount (Last 12 months)

**Total_Trans_Ct :** Total Transaction Count (Last 12 months)

**Total_Ct_Chng_Q4_Q1 :** Change in Transaction Count (Q4 over Q1)

**Avg_Utilization_Ratio :** Average Card Utilization Ratio

In [53]:
credit_card_customers = pd.read_csv(r"pd_explain/Datasets/BankChurners.csv")
credit_card_customers = credit_card_customers.drop([
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",
], axis=1)

credit_card_customers = credit_card_customers.rename(columns={
    "Total_Relationship_Count": "Registered_Products_Count",
    "Total_Revolving_Bal": "Credit_Used",
    "Avg_Open_To_Buy": "Credit_Open_To_Buy",
    "Months_Inactive_12_mon": "Months_Inactive_Count_Last_Year",
    "Contacts_Count_12_mon": "Contacts_Count_Last_Year",
    "Total_Amt_Chng_Q4_Q1": "Total_Amount_Change_Q4_vs_Q1",
    "Total_Trans_Amt": "Total_Transitions_Amount",
    "Total_Trans_Ct": "Total_Transitions_Count",
    "Total_Ct_Chng_Q4_Q1": "Total_Count_Change_Q4_vs_Q1",
    "Avg_Utilization_Ratio": "Credit_Avg_Utilization_Ratio"
})

credit_card_customers.to_csv(r"pd_explain/Datasets/bank_churners_user_study.csv", index=False)


In [ ]:
credit_card_customers.display()

# OPERATION #1

In [ ]:
filter_results = credit_card_customers[credit_card_customers["Customer_Age"] > 45]
explains = filter_results.explain(top_k=5, show_scores=True)

# OPERATION #2

In [ ]:
loyal = credit_card_customers[credit_card_customers.Attrition_Flag == "Existing Customer"]

In [ ]:
churn = credit_card_customers[credit_card_customers.Attrition_Flag != "Existing Customer"]

In [ ]:
x = loyal.Total_Count_Change_Q4_vs_Q1.mean()
filter_results_2 = churn[churn["Total_Count_Change_Q4_vs_Q1"] > x]
filter_results_2

In [ ]:
figs = filter_results_2.explain(top_k=3, figs_in_row = 3)

# OPERATION #3

In [ ]:
churn

In [ ]:
group_by_decade_result = churn.groupby(by=['Marital_Status', 'Gender', 'Education_Level']).agg(
    {"Credit_Limit": ["mean", "max", "min"], "Credit_Used": ["mean", "median"]})
figs = group_by_decade_result.explain(top_k=2)
# start with groups

In [ ]:
figures = group_by_decade_result.explain(top_k=2)

In [ ]:
group_by_decade_result = churn.groupby(["Marital_Status", "Gender", "Education_Level"])['Credit_Used'].mean()



In [ ]:
group_by_decade_result

In [ ]:
figs = group_by_decade_result.explain(top_k = 5, show_scores=True)

# OPERATION #4

In [ ]:
from functools import reduce


def total_more_then_20(series):
    return reduce(lambda x, y: x + 1 if y > 0.2 else x, series, 0)

In [ ]:
group_by_decade_result = churn.groupby(by=['Income_Category', 'Gender', 'Education_Level']).agg(
    {"Credit_Avg_Utilization_Ratio": ['mean', total_more_then_20]})
group_by_decade_result

In [ ]:
group_by_decade_result.explain(show_scores=True)

# JOIN EXAMPLE

In [ ]:
from pd_explain.custom_bins.date_time_bin import bin_datetime
from explain_ed.Measures.Bins import Bins

products = pd.read_csv(r"pd_explain\Datasets\products.csv").rename(columns={'item_no': "item"})
products['list_date'] = pd.to_datetime(products['list_date'], format='%Y-%m-%d %H:%M:%S')
sales = pd.read_csv(r"pd_explain\Datasets\sales.csv")
Bins.register_binning_method(bin_datetime, False)


In [ ]:
sales

In [ ]:
products

In [ ]:
join_results = products.join(sales, on=['item'], how='inner', lsuffix='_x', rsuffix='_y')

In [ ]:
join_results.explain(top_k=2)


# Explain use only columns

In [ ]:
join_results.explain(
    attributes=['products_category_name', 'products_vendor', 'sales_county', 'sales_category_name', 'sales_vendor_no',
                'sales_vendor', 'sales_pack', 'sales_liter_size', 'sales_state_btl_cost', 'sales_btl_price',
                'sales_bottle_qty', 'sales_total'], top_k=3)

# Explain ignore columns

In [ ]:
join_results = products.merge(sales, on=['item'])
join_results

In [ ]:
join_results.explain({"dbl_date": "i", "dbr_date": "i"})